In [2]:
import kickstart as ks

Load data from dataset files.

In [3]:
train = ks.load_data("digits_train.jsonlines")
test = ks.load_data("digits_test.jsonlines")

Count elements from training set.

In [4]:
ctrain = len(train)
print "Elements in training set: {}".format(ctrain)

Elements in training set: 60000


Show type of data.

In [5]:
type(train[0])

dict

Print keys to explore it.

In [6]:
print train[0].keys()

[u'image', u'label']


Let's see how is encoded the 'image'.

In [7]:
type(train[0]['image'])

list

In [8]:
type(train[0]['label'])

unicode

Ok, let's see more:

In [9]:
print "Elements in image feature: {}".format(len(train[0]['image']))
#print train[0]['image']
print "Label: {}".format(train[0]['label'])

Elements in image feature: 784
Label: 0


As we see: 784 = 28x28 (size of picture).

We know that each number is the gray scale between 0 (white) and 255 (black). Let's see the variation of gray scales in the picture. 

In [10]:
def var_gray(image):
    result = [0] * 256
    for idx in xrange(len(image)):
        result[image[idx]] += 1
    return result

gs0 = var_gray(train[0]['image'])
#print "Count for each gray scale:"
#print gs0

Now let's see gray scales that represents "something" (they aren't blur or noise for example).

In [11]:
for i in xrange(len(gs0)):
    if gs0[i]>20:
        print "The gray scale {0} is present {1} times.".format(i, gs0[i])

The gray scale 0 is present 608 times.
The gray scale 252 is present 41 times.
The gray scale 253 is present 23 times.


So, to keep a margin, consider those pixels with gray > 160.

In [12]:
def convert_image(image, margin=160):
    result = [pixel >= margin for pixel in image]
    return result

pic0_converted = convert_image(train[0]['image'])
#print pic0_converted

from itertools import islice

def draw_ascii_image(image):
    it = iter(image)
    for _ in range(28):
        line = []
        for b in islice(it, 28):
            if b:
                c = "#"
            else:
                c = "."
            line.append(c * 2)
        print("".join(line))

draw_ascii_image(pic0_converted)

........................................................
........................................................
........................................................
........................................................
..................................##....................
..............................##########................
............................############................
..........................##########..####..............
......................##############..######............
......................########..####....####............
....................########............####............
..................########..............######..........
................######..................######..........
................####....................######..........
..............######....................######..........
..............####......................####............
..............####......................####............
..............####.............

In [13]:
def mean_number(looked_num):
    global train
    nums = [convert_image(sample['image']) for sample in train if sample['label'] == looked_num]
    nums_count = [num.count(True) for num in nums]
    nums_prom = sum(nums_count)/len(nums_count)
    return nums_prom

means = []
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for num in numbers:
    means.append(mean_number(num))

print means

[125, 54, 106, 100, 86, 90, 98, 81, 107, 87]


Well, that's a very bad way to search for a coincidence. But it's something.

So, now I will build a model for prediction, using that ugly method.

In [14]:
def build_prediction():
    global test, means
    result = []
    for sample in test:
        ct_img = convert_image(sample['image']).count(True)
        def calc_distance(b):
            return abs(ct_img-b)
        distances = list(map(calc_distance, means))

        # Initial value is all differents
        mdist = 784
        candidate = -1
        for idx in xrange(len(distances)):
            if distances[idx] < mdist:
                mdist = distances[idx]
                candidate = idx
        result.append(candidate)
    return result

In [15]:
result = build_prediction()

Let's see our ugly method in action:

In [16]:
import random

for _ in xrange(0, 25):
    idx = random.randint(0, len(test))
    ct_img = convert_image(test[idx]['image'])
    print "Real image: "
    draw_ascii_image(ct_img)
    print "Value predicted: ", result[idx]

Real image: 
........................................................
........................................................
........................................................
........................................................
........................................................
..................................##....................
................##########..########....................
..............####################......................
..............######....................................
..............######....................................
..............####......................................
............####........................................
............####....####################................
............##############################..............
............########..................######............
........................................####............
........................................####............
..................